In [1]:
!pip install -q pandas scikit-learn transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==1

In [2]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
import torch

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define your file paths on Google Drive
base_path = "/content/drive/MyDrive/yelp_data/"
review_path = base_path + "yelp_academic_dataset_review.json"
user_path = base_path + "yelp_academic_dataset_user.json"
biz_path = base_path + "yelp_academic_dataset_business.json"

# Load limited chunk from each to avoid memory overload
review_chunk = pd.read_json(review_path, lines=True, chunksize=100000)
review = next(review_chunk).sample(10000, random_state=42)

user_chunk = pd.read_json(user_path, lines=True, chunksize=50000)
user = next(user_chunk)

biz = pd.read_json(biz_path, lines=True)  # usually smaller

# Merge dataframes
df = review.merge(user, on='user_id', suffixes=('', '_user'))
df = df.merge(biz, on='business_id', suffixes=('', '_biz'))

# Filter and clean
df = df[['text', 'stars', 'review_count', 'average_stars', 'useful', 'funny', 'cool']]
df.dropna(inplace=True)
df = df.sample(min(len(df), 5000), random_state=42)


In [5]:
# Balance the dataset
df_balanced = df.groupby('labels').apply(lambda x: x.sample(n=min(200, len(x)), random_state=42)).reset_index(drop=True)

# TF-IDF with bigrams and more features
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_text = tfidf.fit_transform(df_balanced['text'])

X_meta = df_balanced[['review_count', 'average_stars', 'useful', 'funny', 'cool']].values
scaler = StandardScaler()
X_meta = scaler.fit_transform(X_meta)

from scipy.sparse import hstack
X = hstack([X_text, X_meta])
y = df_balanced['labels'].astype(int)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Deeper MLP
mlp = MLPClassifier(hidden_layer_sizes=(512, 128), alpha=1e-4, max_iter=20, random_state=42)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

# Evaluation
print("=== Improved MLP Classifier Results ===")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=[1,2,3,4,5])
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=[1,2,3,4,5], yticklabels=[1,2,3,4,5])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Improved MLP Confusion Matrix")
plt.show()

=== MLP Classifier Results ===
              precision    recall  f1-score   support

           1       0.57      0.35      0.43        46
           2       0.33      0.06      0.10        49
           3       0.48      0.15      0.23       105
           4       0.43      0.58      0.50       211
           5       0.62      0.76      0.68       255

    accuracy                           0.53       666
   macro avg       0.49      0.38      0.39       666
weighted avg       0.51      0.53      0.49       666



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


In [6]:
!pip install -U transformers

In [7]:
import os
os.environ["WANDB_DISABLED"] = "true"

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Rename stars to labels
df = df.rename(columns={"stars": "labels"})
dataset = Dataset.from_pandas(df[['text', 'labels']])
dataset = dataset.class_encode_column("labels")

tokenized = dataset.map(lambda x: tokenizer(x['text'], truncation=True), batched=True)
tokenized = tokenized.train_test_split(test_size=0.2)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(axis=1) == p.label_ids).mean()}
)

trainer.train()
trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-7-e34e566e3b36>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.992700


{'eval_loss': 0.905337929725647,
 'eval_accuracy': 0.6306306306306306,
 'eval_runtime': 12.9704,
 'eval_samples_per_second': 51.348,
 'eval_steps_per_second': 6.476,
 'epoch': 2.0}